In [1]:
import json 
import phaselocker as pl
import numpy as np 
from bokeh.io import output_notebook, show
import phaselocker.geometry as geom
import phaselocker.sampling as mc 
import phaselocker.visualization as vis
from sklearn.linear_model import BayesianRidge


output_notebook()       # Make our plots appear in the notebook instead of exporting to a file


Loading BokehJS ...

## Load the dataset
We will use a high-fidelity dataset generated using Density Functional Theory (DFT) to train this cluster expansion model. 

In this demo, we will train models that preserve the ground states that are observed in the DFT data set using the ground state order parameter. 

Each atomic configuration provides a constraint on the definition of the ground state cone. Including more uncalculated structures will improve the definition of the ground state cone, so we will include a large number of correlation vectors for atomic configurations that have not yet been calculated with DFT. 

In [2]:
#Load calculated LiAl data
with open('data/calculated_configs.json','r') as f:
    calc_data = json.load(f)
calc_comp = np.array(calc_data['comp'])
calc_corr = np.array(calc_data['corr'])
calc_ef = np.array(calc_data['formation_energy'])

#Load uncalculated data
with open("data/uncalculated_configs.json",'r') as f:
    uncalc_data = json.load(f)
uncalc_comp = np.array(uncalc_data['comp'])
uncalc_corr = np.array(uncalc_data['corr'])

#Combine datasets to make an over-enumerated dataset
all_comp = np.vstack([calc_comp, uncalc_comp])
all_corr = np.vstack([calc_corr, uncalc_corr])


## Identify ground state configurations

Ground state configurations are the vertices of the lower convex hull in composition-formation energy space. They are marked by black diamonds in the plot below. 

We will enforce these ground states so that our cluster expansion models will predict exactly these ground states, and no others within the calculated or uncalculated data sets. 




In [3]:
#View the formation energies and lower convex hull from DFT calculations
p = vis.binary_convex_hull_plotter(comp=calc_comp, observed_energies=calc_ef)
output_notebook()
show(p)

Loading BokehJS ...

In [4]:
#Find the observed ground state incides
observed_hull = geom.full_hull(compositions=calc_comp, energies=calc_ef)
observed_gs_indices, simplex_indices = geom.lower_hull(observed_hull)

#Enforce the observed ground states
imposed_vertices = observed_gs_indices
imposed_simplices = observed_hull.simplices[simplex_indices]
print("Observed indices:       ",observed_gs_indices)
print("Imposing these indices: ",imposed_vertices)

Observed indices:        [  0   1  15  28  49  83 267 441]
Imposing these indices:  [  0   1  15  28  49  83 267 441]


## Load a starting vector

To sample in a subspace that is strictly bounded by infinite-potential walls, we need a valid initial vector that is inside the correct ray-bounded 'cone' within the ECI vector space. 

A candidate vector was found in the other demo notebook "find_cone_with_gradient.ipynb", so we will use that vector.


In [5]:
#Load the in-cone vector, found in "find_cone_with_gradient.ipynb". 
with open('data/final_eci.json','r') as f:
    in_cone_vector = np.array(json.load(f)['eci'])

## Assign meaningful precisions
We want to sample the posterior distribution using a norm-regularized prior, and a cone-restricted prior, with the likelihood function. 

The norm-regularized prior and the likelihood function both need a 'precision'. For a normal distribution, precision is proportional to the inverse of the variance. Precision dictates the length scale of errors, and whether or not they are significant. This also means that they encode uncertainties.  


If you do not already have estimates for noise precision (for the likelihood) and ECI precision, you can use Bayesian Ridge from scikit-learn to estimate them.

In [6]:
#Estimate precisions using scikit learn bayesian ridge regression


#Create a fit object. Do not use fit_intercept=True, as one of the coefficients does this already. 
bayes_ridge = BayesianRidge(fit_intercept=False)
bayes_ridge.fit(calc_corr, calc_ef)


#Scikit learn uses different notation for the regularization parameters.
beta = bayes_ridge.alpha_       #Estimated precision of the noise
alpha = bayes_ridge.lambda_     #Estimated precision of the weights

#Print the precisions as their inverse square roots to give an estimate of the standard deviation in eV/primitive cell units
print("Estimated noise standard deviation: ", 1/np.sqrt(beta)," eV/prim")
print("Estimated model standard deviation: ", 1/np.sqrt(alpha)," eV/prim")

Estimated noise standard deviation:  0.005135005654780729  eV/prim
Estimated model standard deviation:  0.10233608080087712  eV/prim


## Define the prior and likelihood

To sample the posterior distribution, we represent it using a potential energy surface, as a sum of individual energy surfaces of any priors and the likelihood function. Each of these potential functions should accept only an ECI vector, and return a scalar 'energy' or penalty. 

We will define three potentials: 
1) An l_2 norm regularized prior (like ridge regression)
2) A strict ground state prior that enforces exact ground state replication. 
3) A likelihood function to incorporate the observed DFT calculations as training data. 

In [7]:
#Define the necessary potentials
l_2_potential = mc.compose_l_p_norm_potential(order=2, scaling=alpha)
cone_potential = mc.compose_hard_cone_potential(all_comp=all_comp,all_corr=all_corr,observed_vertices=imposed_vertices)
likelihood_potential = mc.compose_likelihood_potential(beta=beta, corr=calc_corr,formation_energies=calc_ef)

#Group the potentials in a list 
potentials = [ cone_potential,l_2_potential, likelihood_potential]


## Run Monte Carlo sampling
Now that the posterior is defined through the list of potentials, it is possible to sample from the posterior distribution. 

Many samples are required to give a useful approximation of the posterior distribution. Additionally, if the initial vector is relatively 'far' from the maximum a-posteriori location, sampling will take some time to travel towards the maximum a-posteriori solution, and these initial steps should be discarded, NOT used. 

The example below will not provide enough samples for proper uncertainty quantification, and is only intended as a code demonstration. 

In [8]:
#Run Monte Carlo sampling
results = mc.metropolis_MC_sampling(initial_site=in_cone_vector, step_size=0.0001,num_loops=100,steps_per_loop=100,potentials=potentials)
print(results.keys())

dict_keys(['samples', 'accept_rate'])


In [9]:
print("Accept rate for each loop: ",results['accept_rate'])

[0.51, 0.31, 0.43, 0.46, 0.51, 0.46, 0.43, 0.53, 0.41, 0.33, 0.25, 0.25, 0.39, 0.18, 0.05, 0.24, 0.23, 0.24, 0.54, 0.32, 0.35, 0.48, 0.37, 0.12, 0.4, 0.34, 0.14, 0.33, 0.33, 0.4, 0.29, 0.22, 0.26, 0.25, 0.29, 0.39, 0.28, 0.19, 0.15, 0.18, 0.15, 0.31, 0.34, 0.42, 0.31, 0.2, 0.24, 0.01, 0.08, 0.39, 0.34, 0.51, 0.56, 0.42, 0.57, 0.34, 0.36, 0.33, 0.46, 0.39, 0.45, 0.27, 0.11, 0.37, 0.46, 0.53, 0.58, 0.55, 0.5, 0.51, 0.7, 0.56, 0.53, 0.42, 0.33, 0.59, 0.51, 0.4, 0.67, 0.48, 0.39, 0.45, 0.68, 0.66, 0.62, 0.62, 0.59, 0.44, 0.36, 0.35, 0.49, 0.45, 0.65, 0.57, 0.6, 0.52, 0.54, 0.66, 0.58, 0.63]


## Visualize results


In [11]:
p = vis.binary_convex_hull_plotter(comp=calc_comp, observed_energies=calc_ef, predicted_energies=calc_corr@np.array(results['samples'])[-1])
output_notebook()
show(p)

Loading BokehJS ...

In [12]:
p = vis.eci_plot(np.array(results['samples']))
show(p)